# Load Data

In [ ]:
import pandas as pd  # Mengimpor library pandas untuk memproses data

# Load dataset non-hoax
real_news = pd.read_csv('cleaned_dataset_scrap.csv')

In [ ]:
real_news

,Text,tagging,label
0,Kejaksaan Agung menetapkan mantan Menteri Perd...,non-hoax,1
1,Israel bawa jasad Yahya Sinwar ke Tel Aviv unt...,non-hoax,1
2,Pengelolaan jalan tol di Ibu Kota Nusantara IK...,non-hoax,1
3,KOMPAScom Artis Meghan Markle yang merupakan i...,non-hoax,1
4,Mahkamah Konstitusi MK memutuskan mengubah amb...,non-hoax,1
...,...,...,...
10362,Jakarta Wakapolda Metro Jaya Brigjen Djati Wiy...,non-hoax,1
10363,Jakarta Lembaga survei Indikator merilis hasil...,non-hoax,1
10364,Jakarta Lembaga survei Indikator merilis hasil...,non-hoax,1
10365,Daftar Isi Cara Cek Quick Count Pilkada Jateng...,non-hoax,1


In [ ]:
# Load dataset hoax
fake_news = pd.read_csv('cleaned_dataset_turnbackhoax_10_cleaned.csv')

# Combining and Shuffling the data

In [ ]:
fake_news

,Text,tagging,label
0,BISA DILIHAT SI ONTA YAMAN NGGAK PEDULI ITU AP...,hoax,0
1,ini bener gasih?? Ya Allah gimna keadaan pa ha...,hoax,0
2,Nenek lampir pemimpin partai banteng bercula s...,hoax,0
3,gerombolan kulup banyak menyusup ke ormas2 isl...,hoax,0
4,M g w ti Di Panggil B4waslu:exclamation: Bvntu...,hoax,0
...,...,...,...
6472,Raja Salman dari arab saudi membawa orang-oran...,hoax,0
6473,"hehe, selalu bisa tersenyum melihat tingkah la...",hoax,0
6474,Pak Jokowi itu menjadi walikota periode pertam...,hoax,0
6475,"Hari ini, Rabu (23/09/2015), nilai tukar rupia...",hoax,0


In [ ]:
# Supaya label lebih terkonfirmasi dengan lebal masing"
fake_news['label'] = 0
real_news['label'] = 1

# Gabung file
data = pd.concat([fake_news, real_news]).sample(frac=1.0)

# Simpen gabungan file
data.to_csv('combined.csv', index=False)

print("Data berhasil dimuat dan file CSV gabungan telah dibuat!")

Data berhasil dimuat dan file CSV gabungan telah dibuat!


In [ ]:
# Cek data hasil kombinasi
data

,Text,tagging,label
4040,Liputan6com Jakarta Ketua Umum Ketum Partai Go...,non-hoax,1
622,TEL AVIV KOMPAScom Pemerintah Israel setuju ge...,non-hoax,1
1242,Perkenalkan saya Bupati Situbondo . Dana donas...,hoax,0
2778,Liputan6com Jakarta Direktur Eksekutif Amnesty...,non-hoax,1
2554,Liputan6com Jakarta Dalam momentum Pilkada 202...,non-hoax,1
...,...,...,...
1346,Ma ruf Amin: ajak orang berharta bantu negara ...,hoax,0
6602,Jakarta Lembaga survei Indikator merilis hasil...,non-hoax,1
3686,China Komunis BIADAB sudah mulai cari masalah ...,hoax,0
783,GANJAR RESMI DIKELUARKAN MEGAWATI USIR GANJAR ...,hoax,0


# Preprocess Function

In [ ]:
# Import modul re untuk bekerja dengan regular expressions
import re

# Import PorterStemmer dari library nltk untuk melakukan stemming pada kata
from nltk.stem import PorterStemmer

In [ ]:
# Inisialisasi PorterStemmer untuk melakukan stemming (menghapus akhiran kata seperti "ed", "ing", dll.)
stemmer = PorterStemmer()

# Fungsi untuk memproses teks menjadi format yang lebih bersih dan standar
def preprocess_text(text):
    # Mengganti karakter selain huruf (a-z atau A-Z) dengan spasi
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Mengubah semua huruf menjadi huruf kecil
    text = text.lower()

    # Melakukan stemming pada setiap kata dalam teks untuk mendapatkan bentuk dasarnya
    text = [stemmer.stem(word) for word in text.split()]

    # Menggabungkan kata-kata hasil stemming menjadi satu string kembali
    text = ' '.join(text)

    # Mengembalikan teks yang sudah diproses
    return text

# Training the Model


## Mengecek Ketersediaan GPU

In [ ]:
import tensorflow as tf
# Mengecek apakah GPU tersedia
if tf.test.is_gpu_available():
    device_name = tf.test.gpu_device_name()  # Mendapatkan nama perangkat GPU jika tersedia
else:
    device_name = 'CPU:0'  # Jika tidak ada GPU, gunakan CPU
print('Menggunakan perangkat:', device_name)

Menggunakan perangkat: CPU:0


## Memproses Data

In [ ]:
# Memproses teks menggunakan fungsi preprocess_text
print("\nMemproses data...")

data['Text'] = data['Text'].apply(preprocess_text)  # Memproses kolom teks


Memproses data...


## Membagi Dataset

In [ ]:
# Membagi dataset menjadi data latih, validasi, dan uji
train_ratio = 0.64  # Proporsi data latih
val_ratio = 0.16  # Proporsi data validasi
test_ratio = 0.2  # Proporsi data uji

print("\nMembagi Data...")

train_data = data.sample(frac=train_ratio, random_state=42)  # Mengambil sampel data latih
remaining_data = data.drop(train_data.index)  # Sisa data untuk validasi dan uji

val_data = remaining_data.sample(frac=val_ratio/(val_ratio+test_ratio), random_state=42)  # Data validasi
test_data = remaining_data.drop(val_data.index)  # Data uji


Membagi Data...


## Tokenisasi dan Konversi ke Tensor

In [ ]:
!pip install transformers
from transformers import BertTokenizer

In [ ]:
# Inisialisasi tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Melakukan tokenisasi dan konversi
print("\nMelakukan Tokenisasi dan Konversi...")

train_inputs = tokenizer(
    list(train_data['Text']),
    truncation=True, padding='max_length', max_length=42, return_tensors='tf'
)
val_inputs = tokenizer(
    list(val_data['Text']),
    truncation=True, padding='max_length', max_length=42, return_tensors='tf'
)
test_inputs = tokenizer(
    list(test_data['Text']),
    truncation=True, padding='max_length', max_length=42, return_tensors='tf'
)

## Mengonversi Label ke Tensor

In [ ]:
# Mengonversi label menjadi tensor TensorFlow
train_labels = tf.convert_to_tensor(list(train_data['label']))
val_labels = tf.convert_to_tensor(list(val_data['label']))
test_labels = tf.convert_to_tensor(list(test_data['label']))

## Mengekstrak Tensor Input

In [ ]:
# Mengekstrak tensor token, segment, dan mask dari input BERT
train_token_tensors = train_inputs['input_ids']
train_segment_tensors = train_inputs['token_type_ids']
train_mask_tensors = train_inputs['attention_mask']

val_token_tensors = val_inputs['input_ids']
val_segment_tensors = val_inputs['token_type_ids']
val_mask_tensors = val_inputs['attention_mask']

test_token_tensors = test_inputs['input_ids']
test_segment_tensors = test_inputs['token_type_ids']
test_mask_tensors = test_inputs['attention_mask']

## Membangun dan Mengompilasi Model

In [ ]:
from transformers import TFBertForSequenceClassification
# Membangun model BERT untuk klasifikasi
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Mengompilasi model
print("\nMengompilasi Model...")

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)  # Optimizer
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')  # Akurasi

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])  # Kompilasi model

## Melatih Model

In [ ]:
# Melatih model BERT
print("\nMelatih Model...")

batch_size = 64  # Ukuran batch
num_epochs = 4  # Jumlah epoch

history = model.fit(
    [train_token_tensors, train_segment_tensors, train_mask_tensors], train_labels,  # Data latih
    batch_size=batch_size, epochs=num_epochs,  # Parameter pelatihan
    validation_data=([val_token_tensors, val_segment_tensors, val_mask_tensors], val_labels)  # Data validasi
)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 6))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Menyimpan Model

In [ ]:
# Menyimpan model yang sudah dilatih
print("\nMenyimpan Model...")

model.save_pretrained('/content/model-hofe')  # Menyimpan model ke direktori

# Evaluasi Model

In [ ]:
# Mengevaluasi model pada set data uji
loss, accuracy = model.evaluate(
    [test_token_tensors, test_segment_tensors, test_mask_tensors],  # Input data uji
    test_labels,  # Label data uji
    batch_size=batch_size  # Ukuran batch
)

# Menampilkan hasil evaluasi
print(f'Test loss: {loss * 100:.3f}%')  # Menampilkan loss dalam persen
print(f'Test accuracy: {accuracy * 100:.3f}%')  # Menampilkan akurasi dalam persen

# Uji Coba Pake Prediksi

In [ ]:
# Import library yang dibutuhkan
from transformers import BertTokenizer, TFBertForSequenceClassification
import transformers as tfs
import tensorflow as tf

In [ ]:
# cek versi transformers

tfs.__version__

In [ ]:
# Memuat model yang sudah disimpan
model = TFBertForSequenceClassification.from_pretrained('/content/model-hofe')

In [ ]:
from transformers import TFBertForSequenceClassification

# Gantilah 'username/my-bert-model' dengan ID repositori kamu
model = TFBertForSequenceClassification.from_pretrained("ajimaulana77/hofe-model")


##  Load Model yang Disimpan

## Menyiapkan Tokenizer

In [ ]:
# Menginisialisasi tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## Meminta Input dari Pengguna

In [ ]:
# Meminta input teks judul berita dari pengguna
input_text = input("\n\n Cantumin text berita: ")

## Preprocessing Teks

In [ ]:
# Melakukan preprocessing pada teks input
preprocessed_text = preprocess_text(input_text)

# Menampilkan teks yang sudah diproses
preprocessed_text

## Tokenisasi Teks yang Sudah Diproses

In [ ]:
# Melakukan tokenisasi pada teks yang sudah diproses
inputs = tokenizer(preprocessed_text, truncation=True, padding='max_length', max_length=42, return_tensors='tf')

# Mengambil tensor input yang diperlukan
token_tensors = inputs['input_ids']
segment_tensors = inputs['token_type_ids']
mask_tensors = inputs['attention_mask']

## Prediksi Menggunakan Model

In [ ]:
# Melakukan prediksi menggunakan model
predictions = model.predict([token_tensors, segment_tensors, mask_tensors])
logits = predictions.logits[0]  # Mengambil logits dari hasil prediksi
probabilities = tf.nn.softmax(logits)  # Menghitung probabilitas menggunakan softmax
predicted_label = tf.argmax(probabilities)  # Menentukan label yang diprediksi

## Menampilkan Hasil Prediksi

In [ ]:
# Menampilkan hasil prediksi berdasarkan label
if predicted_label == 0:
    print("\n*-*-Berita Palsu-*-*")  # Jika label yang diprediksi adalah 0, berarti berita tersebut palsu
else:
    print("\n*-*-Berita Asli-*-*")  # Jika label yang diprediksi adalah 1, berarti berita tersebut asli

# Menampilkan probabilitas berita palsu dan berita asli
print("\nProbabilitas menjadi berita palsu: {:.2%}".format(probabilities[0]))  # Probabilitas berita tersebut palsu
print("Probabilitas menjadi berita asli: {:.2%}".format(probabilities[1]))  # Probabilitas berita tersebut asli

In [ ]:

# Plotting the probabilities
labels = ['Berita Palsu', 'Berita Asli']
prob_values = [probabilities[0], probabilities[1]]

# Create a bar chart
plt.figure(figsize=(6, 4))
plt.bar(labels, prob_values, color=['red', 'green'])
plt.title("Prediksi Probabilitas Berita")
plt.ylabel("Probabilitas")
plt.ylim(0, 1)

# Display the value on top of each bar
for i, v in enumerate(prob_values):
    plt.text(i, v + 0.02, f'{v:.2%}', ha='center', va='bottom', fontweight='bold')

# Show the plot
plt.show()

# Simpen Model jadi Zip

In [ ]:
import shutil
import os

# Ganti 'folder_name' dengan nama folder yang ingin di-zip
folder_to_zip = 'model-hofe'
output_zip_file = 'model-hofe'  # Nama file ZIP yang dihasilkan (tanpa .zip)

# Membuat file ZIP
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

print(f"Folder '{folder_to_zip}' berhasil di-zip menjadi '{output_zip_file}.zip'")


# **Pengetesan Model**

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import shutil
import matplotlib.pyplot as plt

def test_script():
    try:
        # 1. Test Loading Datasets
        print("== Menguji Muatan Dataset ==")
        if not os.path.exists('cleaned_dataset_scrap.csv') or not os.path.exists('cleaned_dataset_turnbackhoax_10_cleaned.csv'):
            raise FileNotFoundError("Dataset tidak ditemukan! Pastikan file 'cleaned_dataset_scrap.csv' dan 'cleaned_dataset_turnbackhoax_10_cleaned.csv' tersedia.")
        real_news = pd.read_csv('cleaned_dataset_scrap.csv')
        fake_news = pd.read_csv('cleaned_dataset_turnbackhoax_10_cleaned.csv')
        print("Dataset berhasil dimuat!")

        # 2. Test Preprocessing Function
        print("\n== Menguji Fungsi Preprocessing ==")
        def preprocess_text_test():
            text = "Ini contoh Kalimat123!"
            result = preprocess_text(text)
            if result != "ini contoh kalimat":
                raise ValueError("Fungsi preprocess_text gagal memproses teks!")
            print("Fungsi preprocess_text berjalan dengan benar.")
        preprocess_text_test()

        # 3. Test Model Training
        print("\n== Menguji Pelatihan Model ==")
        if not os.path.exists('model-hofe'):
            raise FileNotFoundError("Model belum dilatih! Harap latih model terlebih dahulu.")
        print("Model berhasil ditemukan.")

        # 4. Test Prediction
        print("\n== Menguji Prediksi ==")
        input_text = "COVID-19 itu tidak nyata, hanya konspirasi."
        preprocessed_text = preprocess_text(input_text)
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        inputs = tokenizer(preprocessed_text, truncation=True, padding='max_length', max_length=42, return_tensors='tf')
        token_tensors = inputs['input_ids']
        segment_tensors = inputs['token_type_ids']
        mask_tensors = inputs['attention_mask']

        model = TFBertForSequenceClassification.from_pretrained('model-hofe')
        predictions = model.predict([token_tensors, segment_tensors, mask_tensors])
        logits = predictions.logits[0]
        probabilities = tf.nn.softmax(logits)
        predicted_label = tf.argmax(probabilities).numpy()

        print(f"Teks Input: {input_text}")
        print(f"Label Prediksi: {'Berita Asli' if predicted_label == 1 else 'Berita Palsu'}")
        print(f"Probabilitas Berita Palsu: {probabilities[0]:.2%}")
        print(f"Probabilitas Berita Asli: {probabilities[1]:.2%}")

        # 5. Test Zipping Model
        print("\n== Menguji Proses Kompresi Model ==")
        zip_file = 'model-hofe.zip'
        if os.path.exists(zip_file):
            print(f"File ZIP model ditemukan: {zip_file}")
        else:
            raise FileNotFoundError("Model belum dikompres menjadi ZIP!")

        print("\nSemua pengujian berhasil!")

    except Exception as e:
        print(f"Tes gagal dengan error: {str(e)}")

# Jalankan pengujian
test_script()